In [6]:
from torchvision.datasets import CelebA
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

root_dir = 'path/to/celeba/dataset'

# Define the transformation. You might want to transform the images (e.g., resizing, normalization) for your specific task.
transform = transforms.Compose([
    transforms.Resize((64, 64)), # Resize images to 64x64
    transforms.ToTensor(), # Convert images to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalize images
])


celeba_dataset = CelebA(root=root_dir, split='train', transform=transform, download=True)


Files already downloaded and verified


In [14]:
len(celeba_dataset)

162770

In [1]:
import torch
import torch.nn as nn

class CNNAutoencoder(nn.Module):
    def __init__(self):
        super(CNNAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1), # Input channels = 3 (RGB), Output channels = 16
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 7) # No padding
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid() # Using Sigmoid to ensure the output is between 0 and 1
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



In [16]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model instantiation
model = CNNAutoencoder().to(device)

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Number of epochs
num_epochs = 10

total_data = len(celeba_dataset)

batch_size = 64

# Create a DataLoader
data_loader = DataLoader(celeba_dataset, batch_size=batch_size, shuffle=True)
# Training loop
for epoch in range(num_epochs):
    for i, data in tqdm(enumerate(data_loader), total=total_data / batch_size):
        img, _ = data
        img = img.to(device)
        
        # Forward pass
        outputs = model(img)
        loss = criterion(outputs, img)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if not i % 25:
            print(f'Batch {i}, Loss: {loss.item():.4f}')
        
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


  0%|          | 2/2543.28125 [00:00<03:33, 11.90it/s]

Batch 0, Loss: 0.7248


  1%|          | 27/2543.28125 [00:03<07:10,  5.85it/s]

Batch 25, Loss: 0.3685


  2%|▏         | 52/2543.28125 [00:07<06:23,  6.50it/s]

Batch 50, Loss: 0.3147


  3%|▎         | 70/2543.28125 [00:10<07:44,  5.33it/s]